In [4]:
# reference:
#   P. J. E. Peebles
#     The Astrophysical Journal 146 (1966) 542
#   E. W. Kolb and M. S. Turner
#     "The Early Universe" chapter 4
# reference:
#   R. A. Alpher, J. W. Follin and R. C. Herman
#     Physical Review 92 (1953) 1347
#   E. W. Kolb and M. S. Turner
#     "The Early Universe" chapter 4

import numpy as np
from scipy.integrate import solve_ivp,quad
from scipy.special import roots_laguerre
from numpy import pi

hbar = 6.58211899e-22# (Planck constant / 2pi) / MeV s
c = 2.99792458e10# light velocity / cm/s
Grav = 6.70881e-45# gravitaion constant / hbar c^5 MeV^-2
me = 0.510998911# electrom mass / MeV
rad_const = pi*pi/15# radiation constant / (hbar*c)^-3
mp = 938.272013# proton mass / MeV
mn = 939.565346# neutron mass / MeV
tau_n = 885.7# neutron mean lifetime / s
kB = 8.617343e-11# Boltzmann constant / MeV/K
NA = 6.02214179e23# Avogadro number / mol^-1
amu = 931.494003# atomic mass unit / MeV
zeta3 = 1.2020569031595942854# apery number

N = 64 # number of nodes for gaussian quadrature
node,weight = roots_laguerre(N)

def expansion(T0, T1, N_nu=3, n_step=256):
    """ adiabatic expansion of the early universe
    T0 = initial temperature /MeV at time t=0
    T1 = final temperature / MeV
    n_step = number of steps at which data are saved
             with logarithmic interval of T
    N_nu = number of neutrino generation
    return T, T_nu, t where
      T = photon temperature / MeV
      T_nu = neutrino temperature / MeV
      t = age of the universe /sec from T=T0
      each shape (n_step,)
    """
    global a_nu # neutrino radiation constant
    a_nu = rad_const*0.875*N_nu
    T = np.geomspace(T0, T1, n_step)
    s = solve_ivp(expansion_eq, T[[0,-1]], [T0,0], t_eval=T)
    return s.t, s.y[0], s.y[1]

G8P3 = np.sqrt(8*np.pi*Grav/3)/hbar;

def expansion_eq(T, y):
    """
    T  = photon temperature / MeV
    y[0] = T_nu = neutrino temperature / MeV
    y[1] = t = age of the universe from T=T0 / sec
    return [d(T_nu)/dT, dt/dT]
    """
    T_nu = y[0] # neutrino temperature
    E_nu = a_nu * T_nu**4 # neutrino energy density
    E_r = rad_const * T**4 # photon energy density
    P_r = E_r/3 # photon pressure
    c_r = 4*E_r # photon specific heat * T
    E_e, P_e, c_e = electron_gas(T)
    E = E_e + E_r
    P = P_e + P_r
    c = (c_e + c_r)/T
    H = G8P3*np.sqrt(E + E_nu) # dln(a)/dt = expansion rate
    dy = c/(E+P)/3 # -dln(a)/dT = dln(T_nu)/dT
    return [dy*T_nu, -dy/H]

def electron_gas(T):
    """ ideal gas of electrons and positrons
    T = temperature / MeV
    return E,P,c where
      E = energy density of gas
      P = pressure of gas
      c = dE/dlnT = (specific heat of gas)*T
        in units MeV^4/(hbar*c)^3
    """
    a = np.expand_dims(me/T, -1)
    x = node
    x2 = x*x
    y = np.sqrt(x*x + a*a)
    z = np.exp(y)
    f = np.asarray([y, x2/y/3, y*y*z/(z+1)])
    f*= x2/(z+1)*np.exp(x)
    f = np.dot(f, weight) # integral [0,infty]
    f*= T**4*2/np.pi**2 # 4*(4pi)/(2pi)^3,
    # 4 = (electron spin) + (positron spin)
    return f

q = (mn-mp)/me
k = 1/quad(lambda x: np.sqrt(x**2-1)*x*(q-x)**2, 1, q)[0]

def weak_rate(T, T_nu, tau_n=tau_n):
    """ proton-neutron weak interaction
    T = photon temperature / MeV
    T_nu = neutrino temperature / MeV
    tau_n = neutron mean lifetime / sec
    return pn,np where
      pn = proton to neutron conversion rate / sec^-1
      np = neutron to proton conversion rate / sec^-1
    """
    mT = me/T
    a = np.expand_dims(mT, -1)
    b = np.expand_dims(T/T_nu, -1)
    c = np.expand_dims(q*mT, -1)

    x = node
    y = x+a
    z = np.exp(y)
    z1,z2 = np.exp((y+c)*b), np.exp((y-c)*b)
    y1,y2 = (y+c)**2/(z1+1), (y-c)**2/(z2+1)
    f = np.asarray([y1*z + y2*z2, y2*z + y1*z1])
    f *= y*np.sqrt(x*(x+2*a))/(z+1)*np.exp(x)
    f = np.dot(f, weight)# integral [0,infty]
    return f/mT**5*k/tau_n


import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d



In [5]:


class particle:
    def __init__(self, mass, spin, name):
        self.mass = mass
        self.spin = spin
        self.name = name
        self.mass_num = int(np.round(mass/amu))
        
    def __hash__(self):
        return hash((self.mass, self.spin))

proton = particle(938.272013, 2, 'proton')
neutron = particle(939.565346, 2, 'neutron')
deutron = particle(1875.612793, 3, 'deutron')
tritium = particle(2808.920906, 2, 'tritium')
helium3 = particle(2808.391383, 2, 'helium3')
helium4 = particle(3727.379109, 1, 'helium4')
lithium7 = particle(6533.833166, 4, 'lithium7')
beryllium7 = particle(6534.184060, 4, 'beryllium7')

reaction = [
    {'reactor':[neutron,proton], 'product':[deutron], 'use':True},
    {'reactor':[deutron,proton], 'product':[helium3], 'use':True},
    {'reactor':[deutron,deutron], 'product':[helium3,neutron], 'use':True},
    {'reactor':[deutron,deutron], 'product':[tritium,proton], 'use':True},
    {'reactor':[helium3,neutron], 'product':[tritium,proton], 'use':True},
    {'reactor':[tritium,deutron], 'product':[helium4,neutron], 'use':True},
    {'reactor':[helium3,deutron], 'product':[helium4,proton], 'use':True},
    {'reactor':[helium3,helium4], 'product':[beryllium7], 'use':True},
    {'reactor':[helium4,tritium], 'product':[lithium7], 'use':True},
    {'reactor':[beryllium7, neutron], 'product':[lithium7,proton], 'use':True},
    {'reactor':[lithium7, proton], 'product':[helium4,helium4], 'use':True }
]

mask = [r['use'] for r in reaction]
reaction = [{k:r[k] for k in ['reactor','product']}
            for r in reaction if r['use']] # remove unused reactions
element = list({p for r in reaction
                for q in r.values() for p in q})

def reaction_rate(T):
    """
    T = temperature / MeV
    return <sigma v> / cm^3 s^-1
      averaged over Maxwell distribution of v
      where sigma = cross section of reaction 
    reference: M. S. Smith, L. H. Kawano and R. A. Malaney
        The Astrophysical Journal Supplement 85 (1993) 219
    """
    T9 = T/kB*1e-9
    T912 = np.sqrt(T9)
    T932 = T9*T912
    T913 = T9**(1/3)
    T923 = T913**2
    T943 = T923**2
    T953 = T9*T923
    T9f = T9/(1 + 0.1071*T9)
    T9f13 = T9f**(1/3)
    T9f56 = T9f**(5/6)
    T9e = T9/(1 + 0.1378*T9)
    T9e13 = T9e**(1/3)
    T9e56 = T9e**(5/6)
    T9a = T9/(1 + 13.076*T9)
    T9a32 = T9a**1.5
    T9d = T9/(1 + 0.759*T9)
    T9d13 = T9d**(1/3)
    T9d56 = T9d**(5/6)

    return np.asarray([
        # n + p -> d + gamma
        4.742e+4*(1. - .8504*T912 + .4895*T9 - .09623*T932
                  + 8.471e-3*T9*T9 -2.80e-4*T9*T932),
        # p + d -> 3He + gamma
        2.65e+3/T923*np.exp(-3.720/T913)*(
            1. + .112*T913 + 1.99*T923
            + 1.56*T9 + .162*T943 + .324*T953),
        # d + d -> n + 3He
        3.95e+8/T923*np.exp(-4.259/T913)*(
            1. + .098*T913 + .765*T923 + .525*T9
            + 9.61e-3*T943 + .0167*T953),
        # d + d -> p + t
        4.17e+8/T923*np.exp(-4.258/T913)*(
            1. + .098*T913 + .518*T923 + .355*T9
            - .010*T943 - .018*T953),
        # n + 3He -> p + t
    	7.21e+8*(1. - .508*T912 + .228*T9),
        # d + t -> n + 4He
    	1.063e+11/T923*np.exp(-4.559/T913 - (T9/.0754)**2)*(
            1. + .092*T913 - .375*T923 - .242*T9
            + 33.82*T943 + 55.42*T953
            ) + 8.047e+8/T923*np.exp(-0.4857/T9),
        # 3He + d -> 4He + p
    	5.021e+10/T923*np.exp(-7.144/T913 - (T9/.270)**2)*(
            1. + .058*T913 + .603*T923 + .245*T9
            + 6.97*T943 + 7.19*T953
            ) + 5.212e+8/T912*np.exp(-1.762/T9),
        # 3He + 4He -> 7Be + gamma
	4.817e+6/T923*np.exp(-14.964/T913)*(
            1. + .0325*T913 - 1.04e-3*T923 - 2.37e-4*T9
            - 8.11e-5*T943 - 4.69e-5*T953
            ) + 5.938e+6*T9f56/T932*np.exp(-12.859/T9f13),
        # 4He + t -> 7Li + gamma
    	3.032e+5/T923*np.exp(-8.090/T913)*(
            1. + .0516*T913 + .0229*T923 + 8.28e-3*T9
            - 3.28e-4*T943 - 3.01e-4*T953
            ) + 5.109e+5*T9e56/T932*np.exp(-8.068/T9e13),
        # 7Be + n -> 7Li + p
    	2.675e+9*(1. - .560*T912 + .179*T9 - .0283*T932
                  + 2.214e-3*T9*T9 - 6.851e-5*T9*T932
            ) + 9.391e+8*T9a32/T932 + 4.467e+7/T932*np.exp(-0.07486/T9),
        # 7Li + p -> 4He + 4He
    	1.096e+9/T923*np.exp(-8.472/T913) \
        - 4.830e+8*T9d56/T932*np.exp(-8.472/T9d13) \
    	+ 1.06e+10/T932*np.exp(-30.442/T9) \
        + 1.56e+5/T923*np.exp((-8.472/T913) - (T9/1.696)**2)*(
            1. + .049*T913 - 2.498*T923 + .860*T9
            + 3.518*T943 + 3.08*T953
            ) + 1.55e+6/T932*np.exp(-4.478/T9)
        ])[mask]/NA


In [15]:
n_index = element.index(neutron)
p_index = element.index(proton)

N = len(element) # number of chemical elements
M = len(reaction) # number of nuclear reactions

index = np.full((2,2,M), -1, dtype=int) # table of nuclear reactions
bind = np.empty(M) # binding energy
balance = np.empty(M) # balancing factor for backward reaction
A = [e.mass_num for e in element] # mass number of elements
e_name = [e.name for e in element] # name of elements
single = [len(r['product'])==1 for r in reaction] # single product or not

for k,r in enumerate(reaction):
    for i,q in enumerate(r.values()):
        for j,p in enumerate(q):
            index[i,j,k] = element.index(p)
    m = [[p.mass for p in q] for q in r.values()]
    g = [[p.spin for p in q] for q in r.values()]
    bind[k] = np.sum(m[0]) - np.sum(m[1])
    balance[k] = np.prod(g[0])/np.prod(g[1])*(
                 np.prod(m[0])/np.prod(m[1]))**1.5

balance[single] /= (2*np.pi)**1.5*(hbar*c)**3

print(bind)
def initialize(T_init=1e1, T_final=1e-3, N_nu=3, tau_n=tau_n):
    """ thermodynamics of the early universe
    T_init,T_final = temperature / MeV
    N_nu = number of neutrino generation
    tau_n = neutron mean lifetime / sec
    """
    global T0,T1,T,T_nu,time,p_n,n_p,interp
    T0,T1 = T_init, T_final
    T, T_nu, time = expansion(10*T0, T1, N_nu) # cosmic expansion
    p_n, n_p = weak_rate(T, T_nu, tau_n) # proton-neutron conversion rate
    p_n[np.isnan(p_n)] = 0
    n_p[np.isnan(n_p)] = 1/tau_n
    interp = interp1d(time, [T, T_nu, p_n, n_p],
                      'cubic', fill_value='extrapolate')
#initialize()

[ 2.224566  5.493423  3.268857  4.032667  0.76381  17.589244 18.353054
  1.586432  2.466849  1.644227 17.346961]


In [13]:
eta = 6.12e-10
nhc3 = 2.75*eta*2*zeta3/np.pi**2/(hbar*c)**3
def rate(T, T_nu):
    """
    T = temperature / MeV
    T_nu = neutrino temperature / MeV
    return [r1,-r2] (shape (2,M)) where
      r1 = forward reaction rate / sec^-1
      r2 = backward reaction rate / sec^-1
    """
    n = nhc3*T_nu**3 # number density of nucleons / cm^-3
    r = reaction_rate(T)
    r[index[0,0]==index[0,1]] /= 2 # halve rate for identical particles
    r1 = r*n
    r2 = r1*balance*np.exp(-bind/T)
    r2[single] *= T**1.5/n
    return np.asarray([r1, -r2])

In [14]:
rate(T,T_nu)

ValueError: operands could not be broadcast together with shapes (11,256) (11,) 